# [Practice] BERT + LoRA 를 활용하여 Classification 구현하기 (Small Version)

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 BERT 모델을 LoRa를 이용하여 ATIS 데이터셋을 직접만드시고, 훈련하는 테스트를 해봅니다.
모두 작성하고 훈련하신 뒤, wandb report를 업로드 해주세요

- 총 실습시간 : 1시간 20분
    - 훈련 시간 : 40분, colab free 기준
    - 코드 작성 40분

In [1]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [2]:
!pip install evaluate==0.4.5


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!wget -O data.zip "https://www.dropbox.com/scl/fi/vmg4wywro0pnf3hoj0tvv/atis_data.zip?rlkey=n9c5eznbmnvagndv9inmsvrnz&st=kgn9gh6f&dl=1"

--2025-07-17 07:06:05--  https://www.dropbox.com/scl/fi/vmg4wywro0pnf3hoj0tvv/atis_data.zip?rlkey=n9c5eznbmnvagndv9inmsvrnz&st=kgn9gh6f&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc743d1dd1c1f05bbd6964da297b.dl.dropboxusercontent.com/cd/0/inline/Ctp2w8lg8U3vtJOauECMG9mcqiqAHojibO0_DYfPMOfFvJDCbQUl0cqjkv7_rlX1--LnklX_o3rE8Kgzfai6pmEQCC2jpJv6mBZSpd_TI1r1XIL1Xvg4vI6Vp_WJWdbEFT0/file?dl=1# [following]
--2025-07-17 07:06:06--  https://uc743d1dd1c1f05bbd6964da297b.dl.dropboxusercontent.com/cd/0/inline/Ctp2w8lg8U3vtJOauECMG9mcqiqAHojibO0_DYfPMOfFvJDCbQUl0cqjkv7_rlX1--LnklX_o3rE8Kgzfai6pmEQCC2jpJv6mBZSpd_TI1r1XIL1Xvg4vI6Vp_WJWdbEFT0/file?dl=1
Resolving uc743d1dd1c1f05bbd6964da297b.dl.dropboxusercontent.com (uc743d1dd1c1f05bbd6964da297b.dl.dropboxusercontent.com)... 162.125.11.15, 2620:100:6019:15::a

In [4]:
!unzip data.zip

Archive:  data.zip
   creating: test/
  inflating: test/label.txt          
  inflating: test/seq.in             
  inflating: test/seq.out            
   creating: train/
  inflating: train/label.txt         
  inflating: train/seq.in            
  inflating: train/seq.out           
   creating: valid/
  inflating: valid/label.txt         
  inflating: valid/seq.in            
  inflating: valid/seq.out           


## ATIS DATA Processing

In [5]:
import pandas as pd
def load_text(folder):
    texts = []
    with open(folder + '/seq.in') as f:
        for line in f:
            line = line.lstrip().rstrip()
            texts.append(line)
    return texts

def load_label(folder):
    labels = []
    with open(folder + '/label.txt') as f:
        for line in f:
            line = line.lstrip().rstrip()
            labels.append(line)
    return labels


In [6]:
## test
folder = './test'
test_texts, test_labels = load_text(folder), load_label(folder)
test_df = pd.DataFrame( {'text':test_texts, 'label':test_labels} )

In [7]:
## test
folder = './valid'
test_texts, test_labels = load_text(folder), load_label(folder)
valid_df = pd.DataFrame( {'text':test_texts, 'label':test_labels} )

In [8]:
## test
folder = './train'
test_texts, test_labels = load_text(folder), load_label(folder)
train_df = pd.DataFrame( {'text':test_texts, 'label':test_labels} )

In [9]:
cls_list = test_df.label.tolist() + valid_df.label.tolist() + train_df.label.tolist()

In [10]:
classes = list( set(cls_list) )

In [11]:
cls_map = {}
for index, key in enumerate(classes):
    cls_map[key] = index

In [12]:
cls_map

{'atis_ground_service#atis_ground_fare': 0,
 'atis_airline#atis_flight_no': 1,
 'atis_ground_fare': 2,
 'atis_flight': 3,
 'atis_airfare#atis_flight_time': 4,
 'atis_aircraft#atis_flight#atis_flight_no': 5,
 'atis_airfare': 6,
 'atis_airport': 7,
 'atis_flight_no#atis_airline': 8,
 'atis_restriction': 9,
 'atis_meal': 10,
 'atis_ground_service': 11,
 'atis_airfare#atis_flight': 12,
 'atis_abbreviation': 13,
 'atis_airline': 14,
 'atis_flight_time': 15,
 'atis_flight_no': 16,
 'atis_quantity': 17,
 'atis_day_name': 18,
 'atis_flight#atis_airline': 19,
 'atis_distance': 20,
 'atis_capacity': 21,
 'atis_cheapest': 22,
 'atis_flight#atis_airfare': 23,
 'atis_city': 24,
 'atis_aircraft': 25}

In [13]:
test_df

text         label
0    i would like to find a flight from charlotte t...   atis_flight
1    on april first i need a ticket from tacoma to ...  atis_airfare
2    on april first i need a flight going from phoe...   atis_flight
3    i would like a flight traveling one way from p...   atis_flight
4    i would like a flight from orlando to salt lak...   atis_flight
..                                                 ...           ...
888  please find all the flights from cincinnati to...   atis_flight
889  find me a flight from cincinnati to any airpor...   atis_flight
890  i'd like to fly from miami to chicago on ameri...   atis_flight
891  i would like to book a round trip flight from ...   atis_flight
892  find me a flight that flies from memphis to ta...   atis_flight

[893 rows x 2 columns]

In [14]:
test_df['label'] = test_df.label.apply(lambda x : cls_map.get(x))
train_df['label'] = train_df.label.apply(lambda x : cls_map.get(x))
valid_df['label'] = valid_df.label.apply(lambda x : cls_map.get(x))

test_df.to_csv('./test.tsv', sep='\t', index=False, header=False)
train_df.to_csv('./train.tsv', sep='\t', index=False, header=False)
valid_df.to_csv('./validation.tsv', sep='\t', index=False, header=False)

In [15]:
test_df

text  label
0    i would like to find a flight from charlotte t...      3
1    on april first i need a ticket from tacoma to ...      6
2    on april first i need a flight going from phoe...      3
3    i would like a flight traveling one way from p...      3
4    i would like a flight from orlando to salt lak...      3
..                                                 ...    ...
888  please find all the flights from cincinnati to...      3
889  find me a flight from cincinnati to any airpor...      3
890  i'd like to fly from miami to chicago on ameri...      3
891  i would like to book a round trip flight from ...      3
892  find me a flight that flies from memphis to ta...      3

[893 rows x 2 columns]

### Huggingface dataset 으로 로딩하기

In [16]:
import os; os.getcwd()

'/workspace/3-1_Practice_BERT_LORA_SMALL_CLS_ATIS'

In [17]:
import pandas as pd
from datasets import Dataset, DatasetDict

# 1) 파일 경로 지정
train_file_path      = "./train.tsv"
validation_file_path = "./validation.tsv"
test_file_path       = "./test.tsv"

# 2) pandas 로 TSV 읽기 (헤더 없으므로 names 지정)
train_df = pd.read_csv(train_file_path,
                       sep="\t",
                       names=["text", "label"],
                       engine="python")
val_df   = pd.read_csv(validation_file_path,
                       sep="\t",
                       names=["text", "label"],
                       engine="python")
test_df  = pd.read_csv(test_file_path,
                       sep="\t",
                       names=["text", "label"],
                       engine="python")

# 3) HuggingFace DatasetDict 으로 변환
dataset = DatasetDict({
    "train":      Dataset.from_pandas(train_df,  preserve_index=False),
    "validation": Dataset.from_pandas(val_df,    preserve_index=False),
    "test":       Dataset.from_pandas(test_df,   preserve_index=False),
})

# 4) 확인
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4478
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 893
    })
})


In [18]:
from IPython.display import Markdown, display

## 향후 쓰일 여러가지 utility function 준비

In [20]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable : {100 * trainable_params / all_param}%"
    )
    return trainable_params


def compare_param(ori_p, peft_p):
    """
    Compare two parameter numbers
    """
    print(f"\n# Trainable Parameter \nBefore: {ori_p:>14,d} \nAfter:  {peft_p:>14,d} \nPercentage: {round(peft_p / ori_p * 100, 2)}%")


def show_trainable_structure(model):
    """
    Print only the trainable parameters only with size
    """
    num_totals = 0
    for _, param in model.named_parameters():
        if param.requires_grad:
            print( f"{_} \t {param.shape} \t\t {param.numel():,}" )
            num_totals += 1

    print(f"\nTotal Number of Parameter Names : {num_totals:,}")

### Dataset Preprocessing

In [21]:
MODEL_ID = "bert-base-uncased"

### Tokenization

tokenized_datasets 에 미리 dataset 전체를 tokenization 해 둡니다.

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_function(examples):
    return tokenizer(examples["text"],
                     padding="max_length",
                     truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4478 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/893 [00:00<?, ? examples/s]

테스트를 위해 작은 데이터 준비

In [23]:
# -- small dataset (about 40min)
#train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4000))
#test_dataset  = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# -- full dataset (15 hours)
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

## BERT + LoRA 준비

Huggingface 및 PEFT 라이브러리를 이용한 LoRA 셋팅은 크게 아래의 순서로 진행됩니다.

1. Original 모델 준비하기
2. LoRA configuration 준비하기
3. PEFT(LoRA) 화 시킨 Model 준비하기

### Original Model 준비하기

In [25]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=len(classes)
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
ori_p = print_trainable_parameters(model)

trainable params: 109,502,234 || all params: 109,502,234 || trainable : 100.0%


### Model Structure 확인

In [27]:
show_trainable_structure(model)

bert.embeddings.word_embeddings.weight 	 torch.Size([30522, 768]) 		 23,440,896
bert.embeddings.position_embeddings.weight 	 torch.Size([512, 768]) 		 393,216
bert.embeddings.token_type_embeddings.weight 	 torch.Size([2, 768]) 		 1,536
bert.embeddings.LayerNorm.weight 	 torch.Size([768]) 		 768
bert.embeddings.LayerNorm.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.self.query.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.self.query.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.self.key.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.self.key.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.self.value.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.self.value.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.output.dense.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.output.dense.bias 	 torch.Size([768]) 		 768
bert.encod

### Original Model 의 성능 확인

In [28]:
from transformers import TrainingArguments

testing_args = TrainingArguments(
    output_dir='./results',  # 결과 저장 디렉토리
    do_eval=True             # 평가 실행 설정
)

### 평가모듈 준비

In [29]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [30]:
from transformers import Trainer

tester = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=test_dataset,

    # computing metrics (during the training)
    compute_metrics=compute_metrics,
)
res = tester.evaluate()

import pandas as pd
pd.DataFrame([res])

eval_loss  eval_model_preparation_time  eval_accuracy  eval_runtime  \
0   3.662219                       0.0026        0.00112        6.6885   

   eval_samples_per_second  eval_steps_per_second  
0                  133.512                 16.745

아무 훈련없이 BASE 모델의 성능이 위와 같이 나옴을 확인하였습니다.

### LoRA configuration 준비하기

In [31]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # TASK TYPE
    r=8, # Rank
    lora_alpha=1,
    lora_dropout=0.1,
    #target_modules=["query", "value"],  # We can specify target Modules
)

이 [코드](https://github.com/huggingface/peft/blob/main/src/peft/utils/peft_types.py|)에 다음과 같은 Task Type 을 지원함을 알 수 있습니다.

```
class TaskType(str, enum.Enum):
    SEQ_CLS = "SEQ_CLS"
    SEQ_2_SEQ_LM = "SEQ_2_SEQ_LM"
    CAUSAL_LM = "CAUSAL_LM"
    TOKEN_CLS = "TOKEN_CLS"
    QUESTION_ANS = "QUESTION_ANS"
    FEATURE_EXTRACTION = "FEATURE_EXTRACTION"
```

- 이번 실습에서는, Sequcne Classification 을 수행하기 때문에, SEQ_CLS라는 TaskType을 설정합니다.
- 또한 LoRA 의 alpha 값으로 1.0 정도를 셋팅합니다.
- BERT와 LoRA를 함께 사용하는 예제에서 `target_modules`를 명시하지 않으면 LoRA는 기본적으로 모델 내의 모든 적용 가능한 레이어나 모듈에 자동으로 적용되며, 이 경우 LoRA는 모델 내에서 가장 효과적인 부분, 일반적으로는 attention layer(Query, Key, Value)와 Feed-Forward layer적용됩니다.


### PEFT(LoRA) 화 시킨 Model 준비하기

In [32]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [33]:
lora_p = print_trainable_parameters(model)

trainable params: 314,906 || all params: 109,817,140 || trainable : 0.28675487269109357%


In [34]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [35]:
compare_param(ori_p, lora_p)


# Trainable Parameter 
Before:    109,502,234 
After:         314,906 
Percentage: 0.29%


### Fine-Tuning 수행하기

#### Huggingface 의 Trainer 활용하여 훈련하기

In [36]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=8,             # 훈련 에폭 수
    per_device_train_batch_size=16,  # 디바이스 당 훈련 배치 크기
    auto_find_batch_size=True,

    ## ---- Learning Rate ----- ##
    warmup_steps=500,                # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율

    ## ---- GPU & NODES ----- ##
    eval_strategy="epoch",

    ## - Logging & Checkpoint - ##
    output_dir='./results',          # 결과와 체크포인트를 저장할 디렉토리

    logging_steps=10,                # 로그를 기록할 스텝 간격
    #save_steps=1000,                 # 체크포인트를 저장할 스텝 간격
)


In [37]:
trainer = Trainer(
    model=model,
    args=training_args,

    # on Which Dataset
    train_dataset=train_dataset, # < train
    eval_dataset=test_dataset,   # < test

    # computing metrics (during the training)
    compute_metrics=compute_metrics,
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [38]:
import warnings

# huggingface 훈련 중 불필요한 Warning 이 많이 발생함 (epoch 별로)
warnings.filterwarnings("ignore", category=UserWarning)

In [39]:
trainer.train()

TrainOutput(global_step=2240, training_loss=1.1520317769476345, metrics={'train_runtime': 822.243, 'train_samples_per_second': 43.569, 'train_steps_per_second': 2.724, 'total_flos': 9462377378021376.0, 'train_loss': 1.1520317769476345, 'epoch': 8.0})

계속해서 성능이 올라가고 있는 것을 확인할 수 있습니다. 즉, Fine-tuning이 이루어지고 있습니다.

#### LoRA BERT 훈련 model structure

In [40]:
show_trainable_structure(model)

base_model.model.bert.encoder.layer.0.attention.self.query.lora_A.default.weight 	 torch.Size([8, 768]) 		 6,144
base_model.model.bert.encoder.layer.0.attention.self.query.lora_B.default.weight 	 torch.Size([768, 8]) 		 6,144
base_model.model.bert.encoder.layer.0.attention.self.value.lora_A.default.weight 	 torch.Size([8, 768]) 		 6,144
base_model.model.bert.encoder.layer.0.attention.self.value.lora_B.default.weight 	 torch.Size([768, 8]) 		 6,144
base_model.model.bert.encoder.layer.1.attention.self.query.lora_A.default.weight 	 torch.Size([8, 768]) 		 6,144
base_model.model.bert.encoder.layer.1.attention.self.query.lora_B.default.weight 	 torch.Size([768, 8]) 		 6,144
base_model.model.bert.encoder.layer.1.attention.self.value.lora_A.default.weight 	 torch.Size([8, 768]) 		 6,144
base_model.model.bert.encoder.layer.1.attention.self.value.lora_B.default.weight 	 torch.Size([768, 8]) 		 6,144
base_model.model.bert.encoder.layer.2.attention.self.query.lora_A.default.weight 	 torch.Size([8